In [1]:
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
from keras.layers import Dense, Activation, LSTM

In [3]:
from keras.optimizers import RMSprop

In [4]:
from keras.utils.data_utils import get_file

In [5]:
import numpy as np

In [6]:
import random

In [7]:
import sys

In [8]:
path = './kazeno_matasaburo.txt'

In [9]:
bindata = open(path, "rb").read()

In [10]:
text = bindata.decode("shift_jis")

In [11]:
print('Size of text: ', len(text))

Size of text:  33106


In [12]:
chars = sorted(list(set(text)))

In [13]:
print('Total chars:', len(chars))

Total chars: 834


In [14]:
char_indices = dict((c,i) for i,c in enumerate(chars))

In [15]:
indices_char = dict((i,c) for i,c in enumerate(chars))

In [16]:
maxlen = 40

In [17]:
step = 3

In [18]:
sentences = []
next_chars = []

In [19]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [20]:
print('Number of sentences: ',len(sentences))

Number of sentences:  11022


In [21]:
# テキストのベクトル化

In [22]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

In [23]:
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [24]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [25]:
# モデルを定義する

In [26]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

In [27]:
model.add(Dense(len(chars)))

In [28]:
model.add(Activation('softmax'))

In [29]:
optimizer = RMSprop(lr=0.01)

In [30]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [31]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [32]:
for iteration in range(1,60):
    print()
    print('-' *50)
    print('繰り返し回数: ', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text)-maxlen-1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('-----diveristy', diversity)
    
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Seedを生成しました: "' + sentence + '"')
        sys.stdout.write(generated)
    
        for i in range(400):
            x = np.zeros((1,maxlen,len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
        
            generated += next_char
            sentence = sentence[1:] + next_char
        
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
繰り返し回数:  1
Epoch 1/1
11022/11022 [==============================] - 19s 2ms/step - loss: 4.5816

-----diveristy 0.2
----- Seedを生成しました: "、おら一等だぞ。一等だぞ。」とかわるがわる叫びながら大よろこびで門をはいって来た"
、おら一等だぞ。一等だぞ。」とかわるがわる叫びながら大よろこびで門をはいって来たこっていました。
「生生生にはっていました。を生っていましていました。
「あここっていました。を》いないましていましたりいましたりいりいっていました。をはっていました。をここにいましたりにいいていました。
「って、ここっていました。と生におこにいました。をこっていました。」と生こここになこににっいていましていました。のにっていました。をこにないていていました。
「生生はいいここここをこをこになっていていましていいっていっていました。
「こ生にここにいいこここにないいいました。」と生っていました。とこっていていました。にはいていました。の生にはいいました。」と生にここにっていました。を生っていました。
　こ生いここだにないないました。っていました。を生っていました。
「あっていました。に生いなをりいていました。
「あこここここにいました。りっていをこっていていました。っていま

-----diveristy 0.5
----- Seedを生成しました: "、おら一等だぞ。一等だぞ。」とかわるがわる叫びながら大よろこびで門をはいって来た"
、おら一等だぞ。一等だぞ。」とかわるがわる叫びながら大よろこびで門をはいって来た生にもいもいていましたのをはにこりっていてやますがってんこきなりいを神ってこんだってこを佐ってに手るるいいましていました。していました。の待いてってていおにをもっっていましたこっていいこていなこれなここは郎のよをにりだってにいました校ましたるの《で》いにま《をれにるっって来ているふまっていこっていしていました。ころんななをこなっりらころっていましたりもかいだしていっていかるかくをらいまるにのをにちって叫なっていこここににいりろ

「あ高一郎がじちのきの中へ出ついちをました。ばこはずを葉に算めだ来て、だいかうちつのからみんなはいたくは筆をはなっい、まったふをあしてい
「あん生はば一子あすぶませなわがななは三郎にちゃすよにするわしいなのだ。勉ょがって高くはっ

-----diveristy 1.2
----- Seedを生成しました: "ちん箱や、家じゅういっぱいでした。一郎はすばやく帯をして、そして下駄《げた》をは"
ちん箱や、家じゅういっぱいでした。一郎はすばやく帯をして、そして下駄《げた》をはつ、下のこして来てあち太だる吉ぺ《ふ》は来たっまあいまたがれでもテきなく来て、」飛は教ジのきしきうと燃た。じした原のうっ日め出していまるそれか。それはけた洗萄が皮い口水をけて困っとをまと来たを立って一ゃはありなり知らやり、谷たがれからだりけれ郎もいって吸らなじっきるようにぼさて、みんなはまたで一草でにに助っりなっと子らんとえっぞいままいやりじ。
　何かゃどらさい室ころな耕なつ集て言いっしたびまた　と三郎がた》はは信ゃり風上だっぽをしていくは、ゅうはつを書えるかりましたく、ン寸はご読向こらの歩と
「嘉石もいいう、あ、嘉助は-きやぎとなって、今太供はきりがらきぼらやって来た。黒いたがへのをはもめしまたみれをも間そうざらもはんまぐて来るんだらどれを吹こ》か向こうと雷舞ラむの三郎は歩ころ《わせけ》を押ち、やっち栗しふなり、二プ。太人は集ってしだいどもを一草わり立と大ぜののだ一人、所て、手るりを

--------------------------------------------------
繰り返し回数:  5
Epoch 1/1
11022/11022 [==============================] - 19s 2ms/step - loss: 2.6138

-----diveristy 0.2
----- Seedを生成しました: "ふってそれをとめました。庄助は、きせるの火をしずかにそれへうつしました。うしろに"
ふってそれをとめました。庄助は、きせるの火をしずかにそれへうつしました。うしろにはころが三郎はすっているのです。
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　

　五年生はもうとうにおかしなんと、みんなに早っとしていましたが、あたいな。」と言いながら本も言いました。
　すると三郎が、そのにはなったのでした。
　嘉助はこっちにしいって、またはたようにすかのにしてのです。
「ああ、あとはあ、西《から》なあかないや。ありな。」と言いながらもうといさなが、
「あああ、西《かな》な。ああり早おつにつなっていました。
　みんなはまってじって来てから、そのからからもうになりかしましたと、それからみんなに叫びました。
　風はみんなは木はじっとおりして

-----diveristy 1.0
----- Seedを生成しました: "とができないというのでした。それから、はさみ無しの一人まけかち［＃「はさみ無しの"
とができないというのでした。それから、はさみ無しの一人まけかち［＃「はさみ無しの来た。お早といたらみんなの前へんんとうとその遊でにもっとびるが、それを見に思いちもうもました。嘉助が、鼻の前にめれから出のかすたとみんなの、二人はちねす。馬に思いんもあかす。」と言いましたよ。とはさんかりいいの人の子ちろっとにばがれたもするとはくながらんでさかさにころように投たせとからあのかってこいました。
　行ったかすてうわって、もうだんだ馬につともない大んように教室のうでとたがした。
「おら、又三郎がすのか見な。」一郎が言いな。そがら、きのうそしなとうにときとしました。
　嘉助はころそろです。
　ついてはぼっか向こうのらペうさんく気水のとわらました。
　　　　　　　　　　　　　　みこうに思っとは馬の形む、どうく暗郎もまって《かさ》かざあって待っていましたが、草へはは又太郎が嘉ぐつすとぐちみもきのができなり来て来て、大人は先生にざっと立ちなんで、きになりしてしていました。みんな

-----diveristy 1.2
----- Seedを生成しました: "とができないというのでした。それから、はさみ無しの一人まけかち［＃「はさみ無しの"
とができないというのでした。それから、はさみ無しの一人まけかち［＃「はさみ無しのに嘉助がすとすくよのかよまでたた。」
　みんなはじっかりしえながらくちょぶにせんとしると、黒うのかりのうのろさの、日くなに来たとだっぱのほが、
「で叫学校いていて、だしだ。わぶっはチなれや。気さな。あは本《さかなねむってきました。
　とこ

KeyboardInterrupt: 